In [123]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import datetime


In [124]:
data_eth_usd = pd.read_csv("csv files/ETH_USD.csv")
data_eth_usd.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1805 entries, 0 to 1804
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tarih   1805 non-null   object
 1   Şimdi   1805 non-null   object
 2   Açılış  1805 non-null   object
 3   Yüksek  1805 non-null   object
 4   Düşük   1805 non-null   object
 5   Hac.    1804 non-null   object
 6   Fark %  1805 non-null   object
dtypes: object(7)
memory usage: 98.8+ KB


In [125]:
df = data_eth_usd.drop(['Açılış', 'Yüksek', 'Düşük', 'Hac.', 'Fark %'], axis=1)
df


,Tarih,Şimdi
0,01.01.2023,"1.202,54"
1,31.12.2022,"1.195,65"
2,30.12.2022,"1.199,99"
3,29.12.2022,"1.200,80"
4,28.12.2022,"1.189,93"
...,...,...
1800,22.01.2018,"999,37"
1801,21.01.2018,"1.022,74"
1802,20.01.2018,"1.178,29"
1803,19.01.2018,"979,57"


In [126]:
df['Şimdi'] = df['Şimdi'].apply(lambda x: (x.split()[0].replace('.', '')))
df['Şimdi'] = df['Şimdi'].apply(lambda x: float(x.split()[0].replace(',', '.'))/1000)
df

,Tarih,Şimdi
0,01.01.2023,1.20254
1,31.12.2022,1.19565
2,30.12.2022,1.19999
3,29.12.2022,1.20080
4,28.12.2022,1.18993
...,...,...
1800,22.01.2018,0.99937
1801,21.01.2018,1.02274
1802,20.01.2018,1.17829
1803,19.01.2018,0.97957


## LSTM

In [127]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = df['Şimdi']
# choose a number of time steps
n_steps = 7
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.20254 1.19565 1.19999 1.2008  1.18993 1.21155 1.22762] 1.21865
[1.19565 1.19999 1.2008  1.18993 1.21155 1.22762 1.21865] 1.22044
[1.19999 1.2008  1.18993 1.21155 1.22762 1.21865 1.22044] 1.21999
[1.2008  1.18993 1.21155 1.22762 1.21865 1.22044 1.21999] 1.21746
[1.18993 1.21155 1.22762 1.21865 1.22044 1.21999 1.21746] 1.21392
[1.21155 1.22762 1.21865 1.22044 1.21999 1.21746 1.21392] 1.2171800000000002
[1.22762 1.21865 1.22044 1.21999 1.21746 1.21392 1.21718] 1.1678199999999999
[1.21865 1.22044 1.21999 1.21746 1.21392 1.21718 1.16782] 1.1826400000000001
[1.22044 1.21999 1.21746 1.21392 1.21718 1.16782 1.18264] 1.1871500000000001
[1.21999 1.21746 1.21392 1.21718 1.16782 1.18264 1.18715] 1.16667
[1.21746 1.21392 1.21718 1.16782 1.18264 1.18715 1.16667] 1.26626
[1.21392 1.21718 1.16782 1.18264 1.18715 1.16667 1.26626] 1.30724
[1.21718 1.16782 1.18264 1.18715 1.16667 1.26626 1.30724] 1.32073
[1.16782 1.18264 1.18715 1.16667 1.26626 1.30724 1.32073] 1.27498
[1.18264 1.18715 1.16667 1.26626

In [128]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2],df['Şimdi'][3], df['Şimdi'][4], df['Şimdi'][5], df['Şimdi'][6]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)


Epoch 1/200
57/57 [==============================] - 1s 6ms/step - loss: 0.4030
Epoch 2/200
57/57 [==============================] - 0s 6ms/step - loss: 0.0317
Epoch 3/200
57/57 [==============================] - 0s 7ms/step - loss: 0.0206
Epoch 4/200
57/57 [==============================] - 1s 9ms/step - loss: 0.0202
Epoch 5/200
57/57 [==============================] - 0s 7ms/step - loss: 0.0184
Epoch 6/200
57/57 [==============================] - 0s 7ms/step - loss: 0.0211
Epoch 7/200
57/57 [==============================] - 0s 8ms/step - loss: 0.0160
Epoch 8/200
57/57 [==============================] - 0s 7ms/step - loss: 0.0165
Epoch 9/200
57/57 [==============================] - 0s 7ms/step - loss: 0.0147
Epoch 10/200
57/57 [==============================] - 0s 6ms/step - loss: 0.0138
Epoch 11/200
57/57 [==============================] - 0s 6ms/step - loss: 0.0166
Epoch 12/200
57/57 [==============================] - 0s 7ms/step - loss: 0.0141
Epoch 13/200
57/57 [=================

## Daily

In [129]:
x = datetime.datetime.now()
print("Prediction:",round(yhat[0][0]*1000,2), "$ Bitcoin/USD in" ,x.strftime('%x'))

Prediction: 1188.24 $ Bitcoin/USD in 01/01/23


## Weekly

In [130]:
x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2],
                df['Şimdi'][3], df['Şimdi'][4], df['Şimdi'][5], float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_2 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2],
                df['Şimdi'][3], df['Şimdi'][4], float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_3 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2],
                df['Şimdi'][3], float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_4 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2], float(
    yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_5 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], float(yhat_5), float(
    yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_6 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], float(yhat_6), float(yhat_5),
                float(yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_7 = model.predict(x_input, verbose=0)

x_input = array([float(yhat_7), float(yhat_6), float(yhat_5), float(
    yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_8 = model.predict(x_input, verbose=0)


yhat = float(yhat)*1000
yhat_2 = float(yhat_2)*1000
yhat_3 = float(yhat_3)*1000
yhat_4 = float(yhat_4)*1000
yhat_5 = float(yhat_5)*1000
yhat_6 = float(yhat_6)*1000
yhat_7 = float(yhat_7)*1000
yhat_8 = float(yhat_8)*1000


In [131]:
print(f"Monday: {round(yhat,3)} Tuesday : {round(yhat_2,3)} Wednesday: {round(yhat_3,3)} Thursday: {round(yhat_4,3)} Friday: {round(yhat_5,3)}  $ ETH/USD ")


Monday: 1188.235 Tuesday : 1153.526 Wednesday: 1141.997 Thursday: 1142.924 Friday: 1144.806  $ ETH/USD 


## Monthly

In [132]:
import array
exchange = array.array('f', [])

for i in range(len(df)):
    if i % 7 == 0:  # it equals to weekly
        exchange.append(df['Şimdi'][i])


In [133]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange
# choose a number of time steps
n_steps = 7
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.20254 1.21865 1.18264 1.26339 1.27973 1.19335 1.14288] 1.2214100360870361
[1.21865 1.18264 1.26339 1.27973 1.19335 1.14288 1.22141] 1.5710300207138062
[1.18264 1.26339 1.27973 1.19335 1.14288 1.22141 1.57103] 1.5904500484466553
[1.26339 1.27973 1.19335 1.14288 1.22141 1.57103 1.59045] 1.3641899824142456
[1.27973 1.19335 1.14288 1.22141 1.57103 1.59045 1.36419] 1.3059500455856323
[1.19335 1.14288 1.22141 1.57103 1.59045 1.36419 1.30595] 1.3234200477600098
[1.14288 1.22141 1.57103 1.59045 1.36419 1.30595 1.32342] 1.2755199670791626
[1.22141 1.57103 1.59045 1.36419 1.30595 1.32342 1.27552] 1.2942099571228027
[1.57103 1.59045 1.36419 1.30595 1.32342 1.27552 1.29421] 1.3351099491119385
[1.59045 1.36419 1.30595 1.32342 1.27552 1.29421 1.33511] 1.7669899463653564
[1.36419 1.30595 1.32342 1.27552 1.29421 1.33511 1.76699] 1.5788600444793701
[1.30595 1.32342 1.27552 1.29421 1.33511 1.76699 1.57886] 1.426200032234192
[1.32342 1.27552 1.29421 1.33511 1.76699 1.57886 1.4262 ] 1.6182299852371216


In [134]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[exchange[0], exchange[1], exchange[2],exchange[3], exchange[4], exchange[5], exchange[6]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)


Epoch 1/200
8/8 [==============================] - 2s 29ms/step - loss: 1.6532
Epoch 2/200
8/8 [==============================] - 0s 29ms/step - loss: 0.3426
Epoch 3/200
8/8 [==============================] - 0s 30ms/step - loss: 0.2841
Epoch 4/200
8/8 [==============================] - 0s 15ms/step - loss: 0.1885
Epoch 5/200
8/8 [==============================] - 0s 19ms/step - loss: 0.2174
Epoch 6/200
8/8 [==============================] - 0s 10ms/step - loss: 0.1684
Epoch 7/200
8/8 [==============================] - 0s 28ms/step - loss: 0.1620
Epoch 8/200
8/8 [==============================] - 0s 28ms/step - loss: 0.1422
Epoch 9/200
8/8 [==============================] - 0s 15ms/step - loss: 0.1259
Epoch 10/200
8/8 [==============================] - 0s 9ms/step - loss: 0.1127
Epoch 11/200
8/8 [==============================] - 0s 15ms/step - loss: 0.1081
Epoch 12/200
8/8 [==============================] - 0s 10ms/step - loss: 0.1096
Epoch 13/200
8/8 [==============================] 

In [135]:
# onluk olarak yukarı yuvarlandı.
print("Next Month -> Prediction:",round(yhat[0][0]*1000, 3), "$ ETH/USD in", x.strftime('%B'))


Next Month -> Prediction: 1196.763 $ ETH/USD in January


## Yearly


In [136]:
import array
exchange_2 = array.array('f', [])

for i in range(len(df)):
    if i % 30 == 0:  # it equals to weekly
        exchange_2.append(df['Şimdi'][i])


In [137]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange_2
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.20254 1.29569 1.51908 1.3234  1.55757] 1.6069400310516357
[1.29569 1.51908 1.3234  1.55757 1.60694] 1.1340299844741821
[1.51908 1.3234  1.55757 1.60694 1.13403] 1.8061399459838867
[1.3234  1.55757 1.60694 1.13403 1.80614] 2.692850112915039
[1.55757 1.60694 1.13403 1.80614 2.69285] 3.169480085372925
[1.60694 1.13403 1.80614 2.69285 3.16948] 2.492530107498169
[1.13403 1.80614 2.69285 3.16948 2.49253] 3.0134799480438232
[1.80614 2.69285 3.16948 2.49253 3.01348] 3.407210111618042
[2.69285 3.16948 2.49253 3.01348 3.40721] 4.307109832763672
[3.16948 2.49253 3.01348 3.40721 4.30711] 4.611730098724365
[2.49253 3.01348 3.40721 4.30711 4.61173] 3.5592401027679443
[3.01348 3.40721 4.30711 4.61173 3.55924] 3.499269962310791
[3.40721 4.30711 4.61173 3.55924 3.49927] 3.1620500087738037
[4.30711 4.61173 3.55924 3.49927 3.16205] 2.1104800701141357
[4.61173 3.55924 3.49927 3.16205 2.11048] 2.4690799713134766
[3.55924 3.49927 3.16205 2.11048 2.46908] 4.1737799644470215
[3.49927 3.16205 2.11048 2.4690

In [138]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)
# demonstrate prediction
x_input = array(
    [[exchange_2[0], exchange_2[1], exchange_2[2], exchange_2[3], exchange_2[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/300
2/2 [==============================] - 2s 9ms/step - loss: 3.0570
Epoch 2/300
2/2 [==============================] - 0s 8ms/step - loss: 2.4857
Epoch 3/300
2/2 [==============================] - 0s 9ms/step - loss: 1.8766
Epoch 4/300
2/2 [==============================] - 0s 9ms/step - loss: 1.3434
Epoch 5/300
2/2 [==============================] - 0s 8ms/step - loss: 0.8008
Epoch 6/300
2/2 [==============================] - 0s 7ms/step - loss: 0.6398
Epoch 7/300
2/2 [==============================] - 0s 9ms/step - loss: 0.9412
Epoch 8/300
2/2 [==============================] - 0s 8ms/step - loss: 0.7843
Epoch 9/300
2/2 [==============================] - 0s 8ms/step - loss: 0.5542
Epoch 10/300
2/2 [==============================] - 0s 8ms/step - loss: 0.6260
Epoch 11/300
2/2 [==============================] - 0s 8ms/step - loss: 0.6443
Epoch 12/300
2/2 [==============================] - 0s 8ms/step - loss: 0.6376
Epoch 13/300
2/2 [==============================] - 0s 8ms/st

In [139]:
# onluk olarak yukarı yuvarlandı.
print(
    f"Next Year -> Prediction: {round(float(yhat[0][0]*1000),3)}  $ ETH/USD in {x.strftime('%Y')} ")


Next Year -> Prediction: 1823.881  $ ETH/USD in 2023 
